this is folk of the1owls's "Ridge & Huber - 3 Pointer (M)"  with Japanese caption.  
https://www.kaggle.com/the1owl/ridge-huber-3-pointer-m  
just for my understanding

In [1]:
# ライブラリを読み込む
import pandas as pd
import numpy as np
from sklearn import *
import os, glob

C:\Users\oakiyama\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\oakiyama\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\oakiyama\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationW

In [2]:
# データを読み込む
datafiles = sorted(glob.glob('../input/**.csv')) # 全データのパスを取得
# 全データをpd.Dataframeとして読み込んでdictionaryにまとめる
datafiles = {os.path.basename(file)[:-4]: pd.read_csv(file, encoding='latin-1') for file in datafiles}

In [3]:
# 全データをリストアップ
for i, key in enumerate(datafiles):
    print(i, key)

0 TeamSpellings
1 Seasons
2 Conferences
3 RegularSeasonCompactResults
4 NCAATourneySeedRoundSlots
5 NCAATourneySlots
6 ConferenceTourneyGames
7 NCAATourneyCompactResults
8 NCAATourneySeeds
9 MasseyOrdinals
10 Cities
11 TeamConferences
12 GameCities
13 SecondaryTourneyTeams
14 SecondaryTourneyCompactResults
15 SampleSubmissionStage1
16 NCAATourneyDetailedResults
17 Teams
18 TeamCoaches
19 RegularSeasonDetailedResults


In [4]:
"""
SecondaryTourneyというカラムを追加して 'NCAA"もしくは'Regular'の値を与える
SecondaryTourneyはNCAAトーナメント以外のトーナメントのことでNIT, CBI, CIT, V16の４つがある．
ここにNCAA, Regularを追加して試合がどの大会に属するものかを表すカラムとして扱う．
"""
datafiles['NCAATourneyCompactResults']['SecondaryTourney'] = 'NCAA'
datafiles['NCAATourneyDetailedResults']['SecondaryTourney'] = 'NCAA'
datafiles['RegularSeasonCompactResults']['SecondaryTourney'] = 'Regular'
datafiles['RegularSeasonDetailedResults']['SecondaryTourney'] = 'Regular'

In [5]:
### Presets
# カテゴリ型のカラムを数値に置き換える準備
WLoc = {'A': 1, 'H': 2, 'N': 3}
SecondaryTourney = {'NIT': 1, 'CBI': 2, 'CIT': 3, 'V16': 4, 'Regular': 5 ,'NCAA': 6}

In [6]:
# NCAAトーナメントとレギュラーリーグの試合情報を連結

# コンパクト版
# NCAA, レギュラーシーズンの試合情報を連結
games = pd.concat((datafiles['NCAATourneyCompactResults'],datafiles['RegularSeasonCompactResults']), axis=0, ignore_index=True)
# さらにSecondaryTourney () の試合情報を連結
games = pd.concat((games,datafiles['SecondaryTourneyCompactResults']), axis=0, ignore_index=True)

# 詳細版
#games = pd.concat((datafiles['NCAATourneyDetailedResults'],datafiles['RegularSeasonDetailedResults']), axis=0, ignore_index=True)

games.reset_index(drop=True, inplace=True) # indexをリセット
# 試合会場情報をH(ome), A(way), N(eutral)から1, 2, 3 に変換
games['WLoc'] = games['WLoc'].map(WLoc)
# 大会形式情報をNIT, CBI, CIT, V16, Regular, NCAAから1, 2, 3, 4, 5, 6 に変換
games['SecondaryTourney'] = games['SecondaryTourney'].map(SecondaryTourney)
print("games.shape", games.shape)
games.head()

games.shape (154285, 9)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney
0,1985,136,1116,63,1234,54,3,0,6
1,1985,136,1120,59,1345,58,3,0,6
2,1985,136,1207,68,1250,43,3,0,6
3,1985,136,1229,58,1425,55,3,0,6
4,1985,136,1242,49,1325,38,3,0,6


In [7]:
###Add Ids
# カラム ID, IDTeam, Team1, Team2, IDTeam1, IDTeam2 を追加
# Team1, Team2は勝利チームと敗北チームをソートしたものなので入れ替わっている場合有り
games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['IDTeams'] = games.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['Team1'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)
games['Team2'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
games['IDTeam1'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
games['IDTeam2'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
print("games.shape", games.shape)
games.head()

games.shape (154285, 15)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney,ID,IDTeams,Team1,Team2,IDTeam1,IDTeam2
0,1985,136,1116,63,1234,54,3,0,6,1985_1116_1234,1116_1234,1116,1234,1985_1116,1985_1234
1,1985,136,1120,59,1345,58,3,0,6,1985_1120_1345,1120_1345,1120,1345,1985_1120,1985_1345
2,1985,136,1207,68,1250,43,3,0,6,1985_1207_1250,1207_1250,1207,1250,1985_1207,1985_1250
3,1985,136,1229,58,1425,55,3,0,6,1985_1229_1425,1229_1425,1229,1425,1985_1229,1985_1425
4,1985,136,1242,49,1325,38,3,0,6,1985_1242_1325,1242_1325,1242,1325,1985_1242,1985_1325


In [9]:
games[games['ID']=="1985_1234_1116"]

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney,ID,IDTeams,Team1,Team2,IDTeam1,IDTeam2


In [11]:
# 反転版
games2 = games.copy()
games2['ID'] = games['ID'].apply(lambda r: r[:4] + r[9:14] + r[4:9])
games2['IDTeams'] = games['IDTeams'].apply(lambda r: r[5:9] + "_" + r[:4])
games2['Team1'] = games['Team2']
games2['Team2'] = games['Team1']
games2['IDTeam1'] = games['IDTeam2']
games2['IDTeam2'] = games['IDTeam1']

print("games2.shape", games2.shape)
games2.head()

games2.shape (154285, 15)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney,ID,IDTeams,Team1,Team2,IDTeam1,IDTeam2
0,1985,136,1116,63,1234,54,3,0,6,1985_1234_1116,1234_1116,1234,1116,1985_1234,1985_1116
1,1985,136,1120,59,1345,58,3,0,6,1985_1345_1120,1345_1120,1345,1120,1985_1345,1985_1120
2,1985,136,1207,68,1250,43,3,0,6,1985_1250_1207,1250_1207,1250,1207,1985_1250,1985_1207
3,1985,136,1229,58,1425,55,3,0,6,1985_1425_1229,1425_1229,1425,1229,1985_1425,1985_1229
4,1985,136,1242,49,1325,38,3,0,6,1985_1325_1242,1325_1242,1325,1242,1985_1325,1985_1242


In [38]:
games = pd.concat([games, games2])
games.shape

(308570, 21)

In [39]:
###Add Seeds
# シード情報を読み込みIDTeamsとひも付けた形に変換
seeds = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in datafiles['NCAATourneySeeds'].values}
for key in sorted(seeds.keys())[:10]:
    print(key, seeds[key])

1985_1104 7
1985_1112 10
1985_1116 9
1985_1120 11
1985_1130 11
1985_1173 9
1985_1177 10
1985_1181 3
1985_1192 16
1985_1207 1


In [40]:
### Add 2018
# 2018年のシード順位を追加する．2018年のシード順位は2017年と同じとする．
if 2018 not in datafiles['NCAATourneySeeds']['Season'].unique():
    seeds = {**seeds, **{k.replace('2017_','2018_'):seeds[k] for k in seeds if '2017_' in k}}
print("2017年のDuke大学のシード順位:", seeds['2017_1181'], "2018年のDuke大学のシード順位:", seeds['2018_1181'])

2017年のDuke大学のシード順位: 2 2018年のDuke大学のシード順位: 2


In [41]:
# シード順位情報のカラムを追加. 情報がないチームは0を代入
games['Team1Seed'] = games['IDTeam1'].map(seeds).fillna(0)
games['Team2Seed'] = games['IDTeam2'].map(seeds).fillna(0)
games.head()

,DayNum,ID,IDTeam1,IDTeam2,IDTeams,LScore,LTeamID,NumOT,Pred,ScoreDiff,...,Season,SecondaryTourney,SeedDiff,Team1,Team1Seed,Team2,Team2Seed,WLoc,WScore,WTeamID
0,136,1985_1116_1234,1985_1116,1985_1234,1116_1234,54,1234,0,1.0,9.0,...,1985,6,1.0,1116,9.0,1234,8.0,3,63,1116
1,136,1985_1120_1345,1985_1120,1985_1345,1120_1345,58,1345,0,1.0,1.0,...,1985,6,5.0,1120,11.0,1345,6.0,3,59,1120
2,136,1985_1207_1250,1985_1207,1985_1250,1207_1250,43,1250,0,1.0,25.0,...,1985,6,-15.0,1207,1.0,1250,16.0,3,68,1207
3,136,1985_1229_1425,1985_1229,1985_1425,1229_1425,55,1425,0,1.0,3.0,...,1985,6,1.0,1229,9.0,1425,8.0,3,58,1229
4,136,1985_1242_1325,1985_1242,1985_1325,1242_1325,38,1325,0,1.0,11.0,...,1985,6,-11.0,1242,3.0,1325,14.0,3,49,1242


In [42]:
### Additional Features & Clean Up
# 特徴量を追加する
games['ScoreDiff'] = games['WScore'] - games['LScore'] # 点差
# 試合結果 (チーム1 = 勝利チーム なら 1 (勝ち))
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
# 点差の絶対値 (負け試合だと点差がマイナスになっているので)
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
games['SeedDiff'] = games['Team1Seed'] - games['Team2Seed'] # シード順位差
games = games.fillna(-1) # NAを-1で埋める
games.head()

,DayNum,ID,IDTeam1,IDTeam2,IDTeams,LScore,LTeamID,NumOT,Pred,ScoreDiff,...,Season,SecondaryTourney,SeedDiff,Team1,Team1Seed,Team2,Team2Seed,WLoc,WScore,WTeamID
0,136,1985_1116_1234,1985_1116,1985_1234,1116_1234,54,1234,0,1.0,9,...,1985,6,1.0,1116,9.0,1234,8.0,3,63,1116
1,136,1985_1120_1345,1985_1120,1985_1345,1120_1345,58,1345,0,1.0,1,...,1985,6,5.0,1120,11.0,1345,6.0,3,59,1120
2,136,1985_1207_1250,1985_1207,1985_1250,1207_1250,43,1250,0,1.0,25,...,1985,6,-15.0,1207,1.0,1250,16.0,3,68,1207
3,136,1985_1229_1425,1985_1229,1985_1425,1229_1425,55,1425,0,1.0,3,...,1985,6,1.0,1229,9.0,1425,8.0,3,58,1229
4,136,1985_1242_1325,1985_1242,1985_1325,1242_1325,38,1325,0,1.0,11,...,1985,6,-11.0,1242,3.0,1325,14.0,3,49,1242


In [43]:
### Test Set
#　テストデータ作成
sub = datafiles['SampleSubmissionStage1']
sub['WLoc'] = 3 #N NCAAの試合会場は必ず中立
sub['SecondaryTourney'] = 6 #NCAA
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['Season'].astype(int)
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] 
sub.head()

,ID,Pred,WLoc,SecondaryTourney,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,Team2Seed,SeedDiff
0,2014_1107_1110,0.348484,3,6,2014,1107,1110,1107_1110,2014_1107,2014_1110,16,15,1
1,2014_1107_1112,0.564737,3,6,2014,1107,1112,1107_1112,2014_1107,2014_1112,16,1,15
2,2014_1107_1113,0.530011,3,6,2014,1107,1113,1107_1113,2014_1107,2014_1113,16,10,6
3,2014_1107_1124,0.545451,3,6,2014,1107,1124,1107_1124,2014_1107,2014_1124,16,6,10
4,2014_1107_1140,0.530017,3,6,2014,1107,1140,1107_1140,2014_1107,2014_1140,16,10,6


In [44]:
sub.tail()

,ID,Pred,WLoc,SecondaryTourney,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,Team2Seed,SeedDiff
9107,2017_1455_1458,0.516233,3,6,2017,1455,1458,1455_1458,2017_1455,2017_1458,10,8,2
9108,2017_1455_1462,0.288080,3,6,2017,1455,1462,1455_1462,2017_1455,2017_1462,10,11,-1
9109,2017_1457_1458,0.447168,3,6,2017,1457,1458,1457_1458,2017_1457,2017_1458,13,8,5
9110,2017_1457_1462,0.050000,3,6,2017,1457,1462,1457_1462,2017_1457,2017_1462,13,11,2
9111,2017_1458_1462,0.483223,3,6,2017,1458,1462,1458_1462,2017_1458,2017_1462,8,11,-3


In [45]:
### 2017年分を予測
season = 2017
print(season)
# 前年度以前のNCAAのデータを抜き出す
x1 = games[((games['Season']<int(season)) & (games['SecondaryTourney']==6))]
# 同年度以前のNCAA以外のデータを抜き出し追加
x1 = pd.concat((x1,games[((games['Season']<int(int(season)+1)) & (games['SecondaryTourney']!=6))]), axis=0, 
               ignore_index=True)
print(x1.shape)
x1.tail()

2017
(308436, 21)


,DayNum,ID,IDTeam1,IDTeam2,IDTeams,LScore,LTeamID,NumOT,Pred,ScoreDiff,...,Season,SecondaryTourney,SeedDiff,Team1,Team1Seed,Team2,Team2Seed,WLoc,WScore,WTeamID
308431,142,2017_1426_1167,2017_1426,2017_1167,1426_1167,76,1426,0,1.0,4,...,2017,1,0.0,1426,0.0,1167,0.0,1,80,1167
308432,142,2017_1416_1228,2017_1416,2017_1228,1416_1228,58,1228,0,0.0,10,...,2017,1,0.0,1416,0.0,1228,0.0,2,68,1416
308433,148,2017_1210_1167,2017_1210,2017_1167,1210_1167,61,1167,0,0.0,15,...,2017,1,0.0,1210,0.0,1167,0.0,3,76,1210
308434,148,2017_1416_1395,2017_1416,2017_1395,1416_1395,53,1416,0,1.0,15,...,2017,1,0.0,1416,0.0,1395,0.0,3,68,1395
308435,150,2017_1395_1210,2017_1395,2017_1210,1395_1210,56,1210,0,0.0,32,...,2017,1,0.0,1395,0.0,1210,0.0,3,88,1395


In [46]:
# 同年度のNCAAのデータを抜き出す
x2 = games[((games['Season']==int(season)) & (games['SecondaryTourney']==6))]
print(x2.shape)
x2.head()

(134, 21)


,DayNum,ID,IDTeam1,IDTeam2,IDTeams,LScore,LTeamID,NumOT,Pred,ScoreDiff,...,Season,SecondaryTourney,SeedDiff,Team1,Team1Seed,Team2,Team2Seed,WLoc,WScore,WTeamID
2050,134,2017_1243_1448,2017_1243,2017_1448,1243_1448,88,1448,0,1.0,7,...,2017,6,0.0,1243,11.0,1448,11.0,3,95,1243
2051,134,2017_1291_1309,2017_1291,2017_1309,1291_1309,66,1309,0,1.0,1,...,2017,6,0.0,1291,16.0,1309,16.0,3,67,1291
2052,135,2017_1300_1413,2017_1300,2017_1413,1300_1413,63,1300,0,0.0,4,...,2017,6,0.0,1300,16.0,1413,16.0,3,67,1413
2053,135,2017_1344_1425,2017_1344,2017_1425,1344_1425,71,1344,0,0.0,4,...,2017,6,0.0,1344,11.0,1425,11.0,3,75,1425
2054,136,2017_1112_1315,2017_1112,2017_1315,1112_1315,82,1315,0,1.0,18,...,2017,6,-13.0,1112,2.0,1315,15.0,3,100,1112


In [47]:
test = sub[sub['Season']==season]
print(test.shape)
test.head()

(2278, 13)


,ID,Pred,WLoc,SecondaryTourney,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,Team2Seed,SeedDiff
6834,2017_1112_1116,0.489560,3,6,2017,1112,1116,1112_1116,2017_1112,2017_1116,2,8,-6
6835,2017_1112_1124,0.950000,3,6,2017,1112,1124,1112_1124,2017_1112,2017_1124,2,3,-1
6836,2017_1112_1137,0.167036,3,6,2017,1112,1137,1112_1137,2017_1112,2017_1137,2,13,-11
6837,2017_1112_1139,0.760740,3,6,2017,1112,1139,1112_1139,2017_1112,2017_1139,2,4,-2
6838,2017_1112_1153,0.667533,3,6,2017,1112,1153,1112_1153,2017_1112,2017_1153,2,6,-4


In [48]:
sdn = x1.groupby(['IDTeams'], as_index=False)[['ScoreDiffNorm']].mean() # 同じ対戦カードの過去の点差の平均
sdn[:10]

,IDTeams,ScoreDiffNorm
0,1101_1115,8.0
1,1101_1129,-44.0
2,1101_1146,0.6
3,1101_1149,1.0
4,1101_1161,-8.0
5,1101_1170,4.5
6,1101_1179,-17.0
7,1101_1182,-20.0
8,1101_1197,10.0
9,1101_1213,-2.0


In [49]:
test = pd.merge(test, sdn, how='left', on=['IDTeams'])
test['ScoreDiffNorm'] = test['ScoreDiffNorm'].fillna(0.)
test.head()

,ID,Pred,WLoc,SecondaryTourney,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,Team2Seed,SeedDiff,ScoreDiffNorm
0,2017_1112_1116,0.489560,3,6,2017,1112,1116,1112_1116,2017_1112,2017_1116,2,8,-6,0.200000
1,2017_1112_1124,0.950000,3,6,2017,1112,1124,1112_1124,2017_1112,2017_1124,2,3,-1,21.000000
2,2017_1112_1137,0.167036,3,6,2017,1112,1137,1112_1137,2017_1112,2017_1137,2,13,-11,-11.000000
3,2017_1112_1139,0.760740,3,6,2017,1112,1139,1112_1139,2017_1112,2017_1139,2,4,-2,9.666667
4,2017_1112_1153,0.667533,3,6,2017,1112,1153,1112_1153,2017_1112,2017_1153,2,6,-4,6.500000


In [50]:
### Interactions
# Interという特徴量を作る．ここの意味がよく分かっていない
# 直近2年間の試合でチーム1，2両方が戦ったことのある相手に対してチーム1の勝ち数 - 負け数？
"""
チーム1, チーム2, 年度, 結果だけを抜き出す
チーム2を先にした版をまず用意
チーム2が先に来るので結果を反転
"""
inter = games[['IDTeam2','IDTeam1','Season','Pred']].rename(columns={'IDTeam2':'Target','IDTeam1':'Common'})
inter['Pred'] = inter['Pred'] * -1
print(inter.shape)
inter.head()

(308570, 4)


,Target,Common,Season,Pred
0,1985_1234,1985_1116,1985,-1.0
1,1985_1345,1985_1120,1985,-1.0
2,1985_1250,1985_1207,1985,-1.0
3,1985_1425,1985_1229,1985,-1.0
4,1985_1325,1985_1242,1985,-1.0


In [51]:
# チーム1を先にした版を追加
inter = pd.concat((inter,games[['IDTeam1','IDTeam2','Season','Pred']].rename(columns={'IDTeam1':'Target','IDTeam2':'Common'})), axis=0, ignore_index=True).reset_index(drop=True)
print(inter.shape)
inter.head()

(617140, 4)


,Target,Common,Season,Pred
0,1985_1234,1985_1116,1985,-1.0
1,1985_1345,1985_1120,1985,-1.0
2,1985_1250,1985_1207,1985,-1.0
3,1985_1425,1985_1229,1985,-1.0
4,1985_1325,1985_1242,1985,-1.0


In [52]:
# 3年以上前のデータは捨てる
#Only two years back and current regular season
inter = inter[((inter['Season']<=int(season)) & (inter['Season']>int(season)-2))]
print(inter.shape)
inter.head()

(44204, 4)


,Target,Common,Season,Pred
1983,2016_1195,2016_1192,2016,-0.0
1984,2016_1455,2016_1435,2016,-0.0
1985,2016_1380,2016_1221,2016,-1.0
1986,2016_1409,2016_1276,2016,-1.0
1987,2016_1345,2016_1114,2016,-1.0


In [53]:
inter = pd.merge(inter, inter, how='inner', on=['Common','Season'])  # ここがわからん
print(inter.shape)
inter.head()

(2802536, 6)


,Target_x,Common,Season,Pred_x,Target_y,Pred_y
0,2016_1195,2016_1192,2016,-0.0,2016_1195,-0.0
1,2016_1195,2016_1192,2016,-0.0,2016_1437,-0.0
2,2016_1195,2016_1192,2016,-0.0,2016_1200,-0.0
3,2016_1195,2016_1192,2016,-0.0,2016_1389,-1.0
4,2016_1195,2016_1192,2016,-0.0,2016_1343,-0.0


In [54]:
inter = inter[inter['Target_x'] != inter['Target_y']]
print(inter.shape)
inter.head()

(2656000, 6)


,Target_x,Common,Season,Pred_x,Target_y,Pred_y
1,2016_1195,2016_1192,2016,-0.0,2016_1437,-0.0
2,2016_1195,2016_1192,2016,-0.0,2016_1200,-0.0
3,2016_1195,2016_1192,2016,-0.0,2016_1389,-1.0
4,2016_1195,2016_1192,2016,-0.0,2016_1343,-0.0
5,2016_1195,2016_1192,2016,-0.0,2016_1396,-0.0


In [55]:
# inter['ID'] = inter.apply(lambda r:
#                           '_'.join(map(str, [r['Season']+1, 
#                                              r['Target_x'].split('_')[1],
#                                              r['Target_y'].split('_')[1]])), axis=1)
inter['IDTeams'] = inter.apply(lambda r: 
                               '_'.join(map(str, [r['Target_x'].split('_')[1],
                                                  r['Target_y'].split('_')[1]])), axis=1)
inter = inter[['IDTeams','Pred_x']]
print(inter.shape)
inter.head()

(2656000, 2)


,IDTeams,Pred_x
1,1195_1437,-0.0
2,1195_1200,-0.0
3,1195_1389,-0.0
4,1195_1343,-0.0
5,1195_1396,-0.0


In [56]:
inter = inter.groupby(['IDTeams'], as_index=False)[['Pred_x']].sum()
inter = {k:int(v) for k, v in inter.values}
for key in sorted(inter.keys())[:10]:
    print(key, inter[key])

1101_1102 0
1101_1104 0
1101_1107 0
1101_1108 0
1101_1110 0
1101_1111 0
1101_1112 0
1101_1113 0
1101_1114 0
1101_1115 0


In [57]:
# x1 ,x2, test にInterカラムを追加
x1['Inter'] = x1['IDTeams'].map(inter).fillna(0)
x2['Inter'] = x2['IDTeams'].map(inter).fillna(0)
test['Inter'] = test['IDTeams'].map(inter).fillna(0)
print(x1.shape)
x1.head()

(308436, 22)


C:\Users\oakiyama\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,DayNum,ID,IDTeam1,IDTeam2,IDTeams,LScore,LTeamID,NumOT,Pred,ScoreDiff,...,SecondaryTourney,SeedDiff,Team1,Team1Seed,Team2,Team2Seed,WLoc,WScore,WTeamID,Inter
0,136,1985_1116_1234,1985_1116,1985_1234,1116_1234,54,1234,0,1.0,9,...,6,1.0,1116,9.0,1234,8.0,3,63,1116,0.0
1,136,1985_1120_1345,1985_1120,1985_1345,1120_1345,58,1345,0,1.0,1,...,6,5.0,1120,11.0,1345,6.0,3,59,1120,0.0
2,136,1985_1207_1250,1985_1207,1985_1250,1207_1250,43,1250,0,1.0,25,...,6,-15.0,1207,1.0,1250,16.0,3,68,1207,0.0
3,136,1985_1229_1425,1985_1229,1985_1425,1229_1425,55,1425,0,1.0,3,...,6,1.0,1229,9.0,1425,8.0,3,58,1229,0.0
4,136,1985_1242_1325,1985_1242,1985_1325,1242_1325,38,1325,0,1.0,11,...,6,-11.0,1242,3.0,1325,14.0,3,49,1242,0.0


In [58]:
# 説明変数を選択
col = [c for c in x1.columns if c not in [
    'ID', 
    'Team1',
    'Team2', 
    'IDTeams',
    'IDTeam1',
    'IDTeam2',
    'Pred',
    'DayNum', 
    'WTeamID', 
    'WScore', 
    'LTeamID', 
    'LScore', 
    'NumOT', 
    'ScoreDiff']]
col

['ScoreDiffNorm',
 'Season',
 'SecondaryTourney',
 'SeedDiff',
 'Team1Seed',
 'Team2Seed',
 'WLoc',
 'Inter']

In [59]:
sorted(x1['Inter'].unique())

[0.0]

In [60]:
"""
HuberRegressorモデルをトレーニング
HuberRegressorはoutlierにつよいregressorとのこと
"""

reg = linear_model.HuberRegressor()
reg.fit(x1[col], x1['Pred'])
pred = reg.predict(x2[col]).clip(0.05, 0.95)
print('Log Loss:', metrics.log_loss(x2['Pred'], pred))
test['Pred'] = reg.predict(test[col])

Log Loss: 0.307638527445


In [65]:
### Add Validation
results = []
for season in sub['Season'].unique(): # 年度ごとに処理
    print(season)
    # 前年度以前のNCAAのデータを抜き出す
    x1 = games[((games['Season']<int(season)) & (games['SecondaryTourney']==6))]
    # 同年度以前のNCAA以外のデータを抜き出し追加
    x1 = pd.concat((x1,games[((games['Season']<int(int(season)+1)) & (games['SecondaryTourney']!=6))]), axis=0, 
                   ignore_index=True)
    # 同年度のNCAAのデータを抜き出す
    x2 = games[((games['Season']==int(season)) & (games['SecondaryTourney']==6))]
    test = sub[sub['Season']==season]

    sdn = x1.groupby(['IDTeams'], as_index=False)[['ScoreDiffNorm']].mean() # 同じ対戦カードの過去の点差の平均
    test = pd.merge(test, sdn, how='left', on=['IDTeams'])
    test['ScoreDiffNorm'] = test['ScoreDiffNorm'].fillna(0.)
    
#     #Interactions
#     inter = games[['IDTeam2','IDTeam1','Season','Pred']].rename(columns={'IDTeam2':'Target','IDTeam1':'Common'})
#     inter['Pred'] = inter['Pred'] * -1
#     inter = pd.concat((inter,games[['IDTeam1','IDTeam2','Season','Pred']].rename(columns={'IDTeam1':'Target','IDTeam2':'Common'})), axis=0, ignore_index=True).reset_index(drop=True)
#     inter = inter[((inter['Season']<=int(season)) & (inter['Season']>int(season)-2))] #Only two years back and current regular season
#     inter = pd.merge(inter, inter, how='inner', on=['Common','Season'])
#     inter = inter[inter['Target_x'] != inter['Target_y']]
#     #inter['ID'] = inter.apply(lambda r: '_'.join(map(str, [r['Season']+1, r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
#     inter['IDTeams'] = inter.apply(lambda r: '_'.join(map(str, [r['Target_x'].split('_')[1],r['Target_y'].split('_')[1]])), axis=1)
#     inter = inter[['IDTeams','Pred_x']]
#     inter = inter.groupby(['IDTeams'], as_index=False)[['Pred_x']].sum()
#     inter = {k:int(v) for k, v in inter.values}
    
#     x1['Inter'] = x1['IDTeams'].map(inter).fillna(0)
#     x2['Inter'] = x2['IDTeams'].map(inter).fillna(0)
#     test['Inter'] = test['IDTeams'].map(inter).fillna(0)
    col = [c for c in x1.columns if c not in ['ID', 'Team1','Team2', 'IDTeams','IDTeam1','IDTeam2','Pred','DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'ScoreDiff']]
    
    reg = linear_model.HuberRegressor()
    reg.fit(x1[col], x1['Pred'])
    pred = reg.predict(x2[col]).clip(0.05, 0.95)
    print('Log Loss:', metrics.log_loss(x2['Pred'], pred))
    test['Pred'] = reg.predict(test[col])

    results.append(test)

2014
Log Loss: 0.292648526971
2015
Log Loss: 0.319055679494
2016
Log Loss: 0.260309826859
2017
Log Loss: 0.307638527445


In [62]:
results = pd.concat(results, axis=0, ignore_index=True).reset_index(drop=True)
#Testing for Sequence of Scoring
results = {k:float(v) for k,v in results[['ID','Pred']].values}
sub['Pred'] = sub['ID'].map(results).clip(0.05, 0.95).fillna(0.49)
sub[['ID','Pred']].to_csv('rh3p_submission.csv', index=False)